In [1]:
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from random import randint
from PIL import Image,ImageFile

ImageFile.LOAD_TRUNCATED_IMAGES = True

In [2]:
transforms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225]),
    transforms.RandomRotation(randint(1,360)),
    transforms.RandomHorizontalFlip(),

])

In [3]:
train_path = "../Remote sensing paper/Final Dataset/train"
test_path = "../Remote sensing paper/Final Dataset/test"
val_path = "../Remote sensing paper/Final Dataset/val"

In [4]:
def check_image(path):
    try:
        im = Image.open(path)
        return True
    except:
        return False

In [5]:
train_data = torchvision.datasets.ImageFolder(root=train_path,transform=transforms,is_valid_file=check_image)
test_data = torchvision.datasets.ImageFolder(root=test_path,transform=transforms,is_valid_file=check_image)
val_data = torchvision.datasets.ImageFolder(root=val_path,transform=transforms,is_valid_file=check_image)

In [6]:
batch_size = 64
num_classes = 10

In [7]:
train_loader = DataLoader(dataset=train_data,batch_size=batch_size)
test_loader = DataLoader(dataset=test_data,batch_size=batch_size)
val_loader = DataLoader(dataset=val_data,batch_size=batch_size)

In [8]:
if torch.cuda.is_available:
    device = torch.device('cuda')
    torch.cuda.empty_cache()
else:
    device = torch.device('cpu')

In [13]:
def train(model, optimizer, loss_fn, train_loader, val_loader, epochs=2, aux_avai=False, device=device):
    for epoch in range(1, epochs+1):
        training_loss = 0.0
        valid_loss = 0.0
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            inputs, targets = batch
            inputs = inputs.to(device)
            targets = targets.to(device)
            output = model(inputs)
            if aux_avai:
                loss = loss_fn(output, targets) + 0.3*loss_fn(aux1, targets) + 0.3*loss_fn(aux2, targets)
            else:
                loss = loss_fn(output, targets)
            loss.backward()
            optimizer.step()
            training_loss += loss.data.item() * inputs.size(0)
        training_loss /= len(train_loader.dataset)
        
        model.eval()
        num_correct = 0 
        num_examples = 0
        for batch in val_loader:
            inputs, targets = batch
            inputs = inputs.to(device)
            output,_,_ = model(inputs)
            targets = targets.to(device)
            loss = loss_fn(output, targets) 
            valid_loss += loss.data.item() * inputs.size(0)
            correct = torch.eq(torch.max(F.softmax(output, dim=1), dim=1)[1],targets)
            num_correct += torch.sum(correct).item()
            num_examples += correct.shape[0]
        valid_loss /= len(val_loader.dataset)

        print('Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}, accuracy = {:.2f}'.format(epoch, training_loss,
        valid_loss, num_correct / num_examples))

In [18]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'shufflenet_v2_x1_0', pretrained=True)
model.fc = nn.Linear(1024, num_classes)
model.to(device)

Using cache found in C:\Users\manje/.cache\torch\hub\pytorch_vision_v0.10.0


ShuffleNetV2(
  (conv1): Sequential(
    (0): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (stage2): Sequential(
    (0): InvertedResidual(
      (branch1): Sequential(
        (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=24, bias=False)
        (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): Conv2d(24, 58, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (3): BatchNorm2d(58, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (4): ReLU(inplace=True)
      )
      (branch2): Sequential(
        (0): Conv2d(24, 58, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(58, eps=1e-05, momentum=0.1, affine=True, track_running_

In [19]:
optimizer = torch.optim.SGD(params=model.parameters(),lr=0.001,weight_decay=1e-4)

loss = nn.CrossEntropyLoss()
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer=optimizer, step_size=20, gamma=0.1, last_epoch=-1,verbose=True)

c:\Users\manje\anaconda3\envs\torch\Lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


In [ ]:
train(model=model,optimizer=optimizer,loss_fn=loss,train_loader=train_loader,val_loader=val_loader,epochs=100,device=device)